In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install Keras-Applications

In [ ]:
!pip install resnet

In [ ]:
#######################################################
####################    DATASET    ####################
#######################################################

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from os.path import join
import pandas as pd

class Dataset:
    def __init__(self, im_dir):
        # loading training dataset description file
        self.train_list = scipy.io.loadmat('/content/drive/MyDrive/dataset/train_list.mat')

        # loading test dataset description file
        self.test_list = scipy.io.loadmat('/content/drive/MyDrive/dataset/test_list.mat')

        self.im_dir = im_dir

        self.train_dataset_descr, self.validation_dataset_descr = self._create_train_dataset_descr(self.train_list)
        self.test_dataset_descr = self._create_test_dataset_descr(self.test_list)


    def _create_train_dataset_descr(self, examples_list):
      images_paths = []
      images_labels = []
      train_examples_dataframe = None

      validation_images_paths = []
      validation_images_labels = []
      validation_examples_dataframe = None

      for i in range(len(examples_list['file_list'])):
          filename = examples_list['file_list'][i][0][0]

          if i%5 == 0:
            validation_images_paths.append(join(self.im_dir, filename))
          else:
            images_paths.append(join(self.im_dir, filename))

      images_paths = np.array(images_paths)
      validation_images_paths = np.array(validation_images_paths)

      for i in range(len(examples_list['labels'])):
        if i%5 == 0:
          validation_images_labels.append( examples_list['labels'][i][0] )
        else:  
          images_labels.append( examples_list['labels'][i][0] )
      
      images_labels = np.array(images_labels)
      validation_images_labels = np.array(validation_images_labels)

      numpy_table = np.column_stack((images_paths, images_labels))
      validation_numpy_table = np.column_stack((validation_images_paths, validation_images_labels))

      dataframe = pd.DataFrame(data = numpy_table, columns = ['filename', 'breed'])
      validation_dataframe = pd.DataFrame(data = validation_numpy_table, columns = ['filename', 'breed'])

      return (dataframe, validation_dataframe)


    def _create_test_dataset_descr(self, examples_list):
        images_paths = []
        images_labels = []
        examples_dataframe = None

        for i in range(len(examples_list['file_list'])):
            filename = examples_list['file_list'][i][0][0]
            images_paths.append(join(self.im_dir, filename))
        images_paths = np.array(images_paths)

        for i in range(len(examples_list['labels'])):
            images_labels.append( examples_list['labels'][i][0] )
        images_labels = np.array(images_labels)

        numpy_table = np.column_stack((images_paths, images_labels))
        dataframe = pd.DataFrame(data = numpy_table, columns = ['filename', 'breed'])

        return dataframe


In [ ]:
# creating object holding information about dataset (path to used images directory as a parameter)
dataset = Dataset('/content/drive/MyDrive/dataset/Images/')

In [ ]:
#######################################################
#################### DATAGENERATOR ####################
#######################################################

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras

train_datagen_init = ImageDataGenerator( preprocessing_function=keras.applications.resnet50.preprocess_input )
validation_datagen_init = ImageDataGenerator( preprocessing_function=keras.applications.resnet50.preprocess_input )  
test_datagen_init = ImageDataGenerator( preprocessing_function=keras.applications.resnet50.preprocess_input )

train_generator = train_datagen_init.flow_from_dataframe(
    dataset.train_dataset_descr, 
    x_col = 'filename', 
    y_col = 'breed', 
    target_size = (350, 350), 
    color_mode = 'rgb',
    batch_size = 20
)

validation_generator = validation_datagen_init.flow_from_dataframe(
    dataset.validation_dataset_descr, 
    x_col = 'filename', 
    y_col = 'breed', 
    target_size = (350, 350), 
    color_mode = 'rgb',
    batch_size = 20
)

test_generator = test_datagen_init.flow_from_dataframe(
    dataset.test_dataset_descr, 
    x_col = 'filename', 
    y_col='breed', 
    target_size=(350, 350), 
    color_mode='rgb',
    batch_size = 20
)

Found 9600 validated image filenames belonging to 120 classes.
Found 2400 validated image filenames belonging to 120 classes.
Found 8580 validated image filenames belonging to 120 classes.


In [ ]:
#######################################################
################# MODELS ARCHITECTURE #################
#######################################################

from keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
import tensorflow.keras as keras
from keras.models import Model

class ResNet50Util:

  def get_arch(self, weights_type, freezing):
    base_model = ResNet50(include_top = False, weights = weights_type, input_shape=(350, 350, 3))
    base_model.layers.pop()
    base_model.outputs = [base_model.layers[-1].output]
    x=GlobalAveragePooling2D()(base_model.output)
    x=Dense(120, activation='softmax')(x)
    
    model=Model(base_model.inputs, x)

    if freezing == "partial_freezing":
      for layer in base_model.layers[:-26]:
        layer.trainable = False
    elif freezing == "full_freezing":
      for layer in base_model.layers:
        layer.trainable = False

    model.compile(loss=keras.losses.categorical_crossentropy,
                            optimizer='sgd',
                            metrics=['accuracy'])

    model.summary()

    return model

  
  def get_ext_arch(self, weights_type, freezing):
    base_model = ResNet50(include_top = False, weights = weights_type, input_shape=(350, 350, 3))
    base_model.layers.pop()
    base_model.outputs = [base_model.layers[-1].output]
    x=GlobalAveragePooling2D()(base_model.output)
    x=BatchNormalization()(x)
    x=Dropout(0.4)(x)
    x=Dense(1000, activation='relu')(x)
    x=Dropout(0.4)(x)
    x=Dense(120, activation='softmax')(x)
    
    model=Model(base_model.inputs, x)

    if freezing == "partial_freezing":
      for layer in base_model.layers[:-26]:
        layer.trainable = False
    elif freezing == "full_freezing":
      for layer in base_model.layers:
        layer.trainable = False

    model.compile(loss=keras.losses.categorical_crossentropy,
                            optimizer='sgd',
                            metrics=['accuracy'])

    model.summary()

    return model


In [ ]:
#######################################################
############### FIT FUNCTION PARAMETERS ###############
#######################################################

step_size_for_train = train_generator.n // train_generator.batch_size 
step_size_for_validation = validation_generator.n // validation_generator.batch_size
step_size_for_test = test_generator.n // test_generator.batch_size

fit_callbacks = [
    keras.callbacks.CSVLogger(
        filename="/content/drive/MyDrive/logs/LOG_model_ResNet50tl.h5",
        separator=',', 
        append=True
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/MyDrive/logs/WEIGHTS_{epoch:02d}-{val_loss:.2f}.h5", 
        monitor='val_loss', 
        verbose=0, 
        save_best_only=False,
        save_weights_only=True,
        save_freq='epoch', 
        options=None
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5,
        verbose=1,
        mode='auto',
        cooldown=1,
        min_lr=0.00001
    ),
    keras.callbacks.TensorBoard(
        log_dir='/content/drive/MyDrive/logs/tensor_board_logs', 
        histogram_freq=1,
        write_graph=True,
        write_images=True,
        update_freq='epoch',
    )
]

In [ ]:
#######################################################
################### TRAINING MODELS ###################
#######################################################

resnet50_util = ResNet50Util()

In [ ]:
# RANDOM WEIGHTS - MODEL A

model_1A = resnet50_util.get_arch(None, None)

model_1A.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# RANDOM WEIGHTS - MODEL B

model_1B = resnet50_util.get_ext_arch(None, None)

model_1B.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# TRANSFER LEARNING WITHOUUT LAYERS FREEZING - MODEL A

model_2A = resnet50_util.get_arch('imagenet', None)

model_2A.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# TRANSFER LEARNING WITHOUUT LAYERS FREEZING - MODEL B

model_2B = resnet50_util.get_ext_arch('imagenet', None)

model_2B.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# TRANSFER LEARNING WITH PARTIAL LAYERS FREEZING - MODEL A

model_3A = resnet50_util.get_arch('imagenet', 'partial_freezing')

model_3A.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# TRANSFER LEARNING WITH PARTIAL LAYERS FREEZING - MODEL B

model_3B = resnet50_util.get_ext_arch('imagenet', 'partial_freezing')

model_3B.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# TRANSFER LEARNING WITH ALL LAYERS FROZEN - MODEL A

model_4A = resnet50_util.get_arch('imagenet', 'full_freezing')

model_4A.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
# TRANSFER LEARNING WITH ALL LAYERS FROZEN - MODEL B

model_4B = resnet50_util.get_ext_arch('imagenet', 'full_freezing')

model_4B.fit(
    train_generator,
    steps_per_epoch=step_size_for_train,
    validation_data=validation_generator,
    validation_steps=step_size_for_validation,
    epochs=30,
    verbose=1,
    callbacks=fit_callbacks
)

In [ ]:
#######################################################
################## EVALUATING MODELS ##################
#######################################################

In [ ]:
# RANDOM WEIGHTS - MODEL A

model_1A.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# RANDOM WEIGHTS - MODEL B

model_1B.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# TRANSFER LEARNING WITHOUUT LAYERS FREEZING - MODEL A

model_2A.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# TRANSFER LEARNING WITHOUUT LAYERS FREEZING - MODEL B

model_2B.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# TRANSFER LEARNING WITH PARTIAL LAYERS FREEZING - MODEL A

model_3A.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# TRANSFER LEARNING WITH PARTIAL LAYERS FREEZING - MODEL B

model_3B.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# TRANSFER LEARNING WITH ALL LAYERS FROZEN - MODEL A

model_4A.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
# TRANSFER LEARNING WITH ALL LAYERS FROZEN - MODEL B

model_4B.evaluate(
    test_generator,
    steps=step_size_for_test,
    verbose=1
)

In [ ]:
#######################################################
###################### PREDICTION #####################
#######################################################

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

def load_image(path):
  img = image.load_img(path, target_size=(350, 350))
  rslt_img = image.img_to_array(img)
  rslt_img = np.expand_dims(rslt_img, axis=0)
  rslt_img = preprocess_input(rslt_img)

  return rslt_img


def predict(img_path, model):
  image = load_image(img_path)
  solution = model.predict(image)
  print(solution)


predict('/content/drive/MyDrive/dataset/Images/n02085620-Chihuahua/n02085620_7.jpg', model_2A)

In [ ]:
#######################################################
###################### TENSORBOARD ####################
#######################################################

%load_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/logs/tensor_board_logs'